In [1]:
from scipy.spatial import distance

In [2]:
frames = [[(520,122), (211,211),(910,56)], 
          [(522,110), (921,95)], 
          [(200,196),(890,66)], 
          [(910,56),(56,502)], 
          [(919,57),(66,508)], 
          [(920,46),(59,512)],
          [(920,46),(59,512),(520,122)]]

In [198]:
def for_first_frame(boxes,id_details, frame):
    '''
    This function assigns ids to all detected number of bounding boxes found in first frame of the video.
    takes input of location of the bounding box.
    returns dictionary in which key is centroid of the bounding box and value is assigned id.
    '''
    temp_id = 0
    id_list = {}
    for box in boxes:
        centroid = str(box[0]) + "," + str(box[1])
        id_list[centroid] = temp_id
        id_details.append({"id":temp_id, "last_seen_on_frame": frame, "previous_centroid": centroid})
        temp_id += 1
    return id_list, temp_id, id_details

In [199]:
def find_least_distance(centroid, id_, box, distance_to_all_points, least_distance, temp_distance_keeper, new_ids, id_details, frame):
    '''
    This function is responsible for finding nearest previous centroid of the current bounding box's centroid.
    Taks input as 
    centroid                 | string : centroids of the previous bounding boxes
    id_                      | int : id assigned to previous centroid
    box                      | tuple : location of current bounding box
    distance_to_all_points   | list of distance calculated to all the previous centroids
    least_distance           | int: least distance among previously calculated distances with all the previous centroids for the current bounding box
    temp_distance_keeper     | list of dictionalry which holds id, distance, current_centroids, previous_centroid 
                             | (This will be helpful when we will have more that one points having assigned same id)
    new_ids                  | List of assigned ids for each bounding boxes till now.
    '''
    
    centroids = centroid.split(',')
    pc1 = float(centroids[0])
    pc2 = float(centroids[1])
    d = distance.euclidean((pc1, pc2),(box[0],box[1]))
    distance_to_all_points.append(d)
    key_val = str(box[0]) + "," + str(box[1])
    if d<least_distance:
        least_distance = d
        tmp_assigned_id = id_
        same_id_found = 0
        for tmp_dist in temp_distance_keeper:
            '''
            This checks if id is already assigned before, if id is assigned then check the distance of the,
            2 centroids and chooses the point which have least distance.
            '''
            if tmp_assigned_id == tmp_dist["id"]:
                same_id_found = 1
                if least_distance < tmp_dist["distance"]:
                    tmp_dist["distance"] = least_distance
                    tmp_dist["current_point"] = key_val
                    new_ids[key_val] = tmp_dist["id"]
        if not same_id_found:
            temp_distance_keeper.append({"id":tmp_assigned_id, "distance":least_distance, "current_point":key_val, " previous_point ":centroid})
            new_ids[key_val] = tmp_assigned_id

    return new_ids, temp_distance_keeper, distance_to_all_points, least_distance, id_details

In [205]:
def find_nearest_point(boxes, previous_centroids, last_used_id, id_details, frame):
    Ids = previous_centroids["id"]
    previous_ids = previous_centroids["id"].values()
    temp_distance_keeper = []
    new_ids = {}
    
    for box in boxes:
        distance_to_all_points = []
        least_distance = 100000000000000
        tmp_assigned_id = None
        print("\n")
        key_val = str(box[0]) + "," + str(box[1])
        for centroid, id_ in Ids.items():
            new_ids, temp_distance_keeper, distance_to_all_points, least_distance, id_details = find_least_distance(centroid, id_, box, distance_to_all_points, least_distance, temp_distance_keeper, new_ids, id_details, frame)
        new_ids, last_used_id, id_details = register_new_id(distance_to_all_points, new_ids, last_used_id, key_val, id_details, frame)
    
    update_id_details(id_details, new_ids, frame)
    return new_ids, last_used_id, id_details

In [210]:
def update_id_details(id_details, new_ids, frame):
    for centroid, id_ in new_ids.items():
        id_details[id_]["previous_centroid"] = centroid
        id_details[id_]["last_seen_on_frame"] = frame
    return id_details

In [211]:
def register_new_id(distance_to_all_points, new_ids, last_used_id, centroids, id_details, frame):
    """
    If the centroid is having distancing from all the points greter than some threshold,
    then this function will add new ID to that bounding box.
    """
    
    all_points = len(distance_to_all_points)
    counter_for_adding_new_id = 0
    index = 0
    for prev_dist in distance_to_all_points:
        if prev_dist > 150:
            counter_for_adding_new_id += 1
        index += 1
    if all_points == counter_for_adding_new_id:
        fount_fluctuation, new_ids, id_details = check_for_id_fluctualtion(id_details, centroids, new_ids, frame)
        if not fount_fluctuation:
            new_ids[centroids] = last_used_id
            id_details.append({'id': last_used_id, 'last_seen_on_frame': frame, 'previous_centroid': centroids})
            last_used_id += 1
    
    return new_ids, last_used_id, id_details

In [212]:
def check_for_id_fluctualtion(id_details, key_val, new_ids, frame):
    print("Before Adding a new id we have to check for ID fluctuation................")
    #print("ID DETAILS - ",id_details)
    distances_from_previous_idd_centroids = []
    least_distance = 100000000000000
    index = 0
    for id_d in id_details:
        previous_centroids = id_d["previous_centroid"].split(',')
        last_seen_frame = id_d["last_seen_on_frame"]
        frame_difference = frame - last_seen_frame
        prv_c1 = float(previous_centroids[0])
        prv_c2 = float(previous_centroids[1])
        current_centroid = key_val.split(",")
        crnt_c1 = float(current_centroid[0])
        crnt_c2 = float(current_centroid[1])
        d = distance.euclidean((prv_c1, prv_c2),(crnt_c1,crnt_c2))
        if d<least_distance and frame_difference < 10:
            least_distance = d
            temp_assigned_id = id_d["id"]
            index_of_assigned_id = index
        index += 1
    if least_distance > 150:
        return False ,new_ids, id_details
    
    new_ids[key_val] = temp_assigned_id
    return True, new_ids, id_details

In [213]:
Ids = []
id_details = [
    #{"id":0,"class":"person","last_seen_on_frame":0, "previous_identified_centroid":"736.8079447746277,259.6186065673828"},
    #{"id":1,"class":"person","last_seen_on_frame":0, "previous_identified_centroid":"656.2119626998901,1055.357551574707"},
]
previous_id = 0
for i in range(0,7):
    print("----- I : ",i)
    if i == 0:
        id_list, previous_id, id_details = for_first_frame(frames[i], id_details, i)
        Ids.append({"frame":i, "id":id_list})
        print(id_details)
        print(Ids)
    else:
        id_list, previous_id, id_details = find_nearest_point(frames[i], Ids[i-1], previous_id, id_details, i)
        Ids.append({"frame":i, "id":id_list})
        print("*************\nFinal Id Details : ",id_details)
        print()
        print(Ids)

----- I :  0
[{'id': 0, 'last_seen_on_frame': 0, 'previous_centroid': '520,122'}, {'id': 1, 'last_seen_on_frame': 0, 'previous_centroid': '211,211'}, {'id': 2, 'last_seen_on_frame': 0, 'previous_centroid': '910,56'}]
[{'frame': 0, 'id': {'520,122': 0, '211,211': 1, '910,56': 2}}]
----- I :  1




*************
Final Id Details :  [{'id': 0, 'last_seen_on_frame': 1, 'previous_centroid': '522,110'}, {'id': 1, 'last_seen_on_frame': 0, 'previous_centroid': '211,211'}, {'id': 2, 'last_seen_on_frame': 1, 'previous_centroid': '921,95'}]

[{'frame': 0, 'id': {'520,122': 0, '211,211': 1, '910,56': 2}}, {'frame': 1, 'id': {'522,110': 0, '921,95': 2}}]
----- I :  2


Before Adding a new id we have to check for ID fluctuation................


*************
Final Id Details :  [{'id': 0, 'last_seen_on_frame': 1, 'previous_centroid': '522,110'}, {'id': 1, 'last_seen_on_frame': 2, 'previous_centroid': '200,196'}, {'id': 2, 'last_seen_on_frame': 2, 'previous_centroid': '890,66'}]

[{'frame': 0, 'id':

In [214]:
Ids

[{'frame': 0, 'id': {'520,122': 0, '211,211': 1, '910,56': 2}},
 {'frame': 1, 'id': {'522,110': 0, '921,95': 2}},
 {'frame': 2, 'id': {'200,196': 1, '890,66': 2}},
 {'frame': 3, 'id': {'910,56': 2, '56,502': 3}},
 {'frame': 4, 'id': {'919,57': 2, '66,508': 3}},
 {'frame': 5, 'id': {'920,46': 2, '59,512': 3}},
 {'frame': 6, 'id': {'920,46': 2, '59,512': 3, '520,122': 0}}]

In [184]:
id_details

[{'id': 0, 'last_seen_on_frame': 6, 'previous_centroid': '920,46'},
 {'id': 1, 'last_seen_on_frame': 6, 'previous_centroid': '59,512'},
 {'id': 2, 'last_seen_on_frame': 1, 'previous_centroid': '921,95'},
 {'id': 3, 'last_seen_on_frame': 3, 'previous_centroid': '56,502'},
 {'id': 4, 'last_seen_on_frame': 6, 'previous_centroid': '520,122'}]